# North Carolina Crop Data

Transform nationwide crop data since 1866 into a lean dataframe with one row per year and Annual production figures for columns.

North Carolina only

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
# read in crop data - 22 million rows nationwide from 1866 to 2024. Some data is weekly and some data is at the county level

folder = 'C:/Users/szums/usdaData/'

df = pd.read_csv(folder + 'qs.crops_20240815.txt',delimiter='\t') 
# ignore the warning about dtypes - those columns will be removed anyway

print(f'{len(df)} rows read.')

C:\Users\szums\AppData\Local\Temp\ipykernel_13852\1696744811.py:5: DtypeWarning: Columns (19,20,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder + 'qs.crops_20240815.txt',delimiter='\t')


22397301 rows read.


In [3]:
# Pare the data down to just the rows we need (state level, annual totals, production, actuals vs forecast

df1 = df[(df['AGG_LEVEL_DESC'] == 'STATE') & (df['STATE_ALPHA'] == 'NC') & (df['SOURCE_DESC'] == 'SURVEY') & (df['REFERENCE_PERIOD_DESC'] == 'YEAR')][['SHORT_DESC','YEAR','VALUE']]

print(f'pared down rows and columns to just {len(df1)} rows.')

df1.info()


pared down rows and columns to just 22723 rows.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 22723 entries, 210 to 22397290
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SHORT_DESC  22723 non-null  object
 1   YEAR        22723 non-null  int64 
 2   VALUE       22723 non-null  object
dtypes: int64(1), object(2)
memory usage: 710.1+ KB


In [4]:

# convert the VALUE column to numeric

df1['VALUE'] = df1['VALUE'].apply(lambda v: None if v[0] == "(" else v)

df1['VALUE'] = df1['VALUE'].str.replace(',','')

df1['VALUE'] = pd.to_numeric(df1['VALUE'])

df1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 22723 entries, 210 to 22397290
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SHORT_DESC  22723 non-null  object 
 1   YEAR        22723 non-null  int64  
 2   VALUE       19924 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 710.1+ KB


In [5]:
# drop the rows that have value zero

df2 = df1[df1['VALUE'] > 0]

df2.info()

# now we are down to just 19,000 rows (from 22 million)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19006 entries, 210 to 22397290
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SHORT_DESC  19006 non-null  object 
 1   YEAR        19006 non-null  int64  
 2   VALUE       19006 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 593.9+ KB


# Reshape the data - one row per year - one column per measure

In [6]:

# Utility functions defined here, used to reshape the data

def add_column(dfx,short_desc,value_name):

    retval = dfx[dfx['SHORT_DESC'] == short_desc][['YEAR','VALUE']].reset_index(drop=True)
    retval.rename(columns={'VALUE':value_name},inplace=True)
    return  retval


def append_column(aggregatingdf,newcol):
    aggregatingdf = pd.merge(aggregatingdf,newcol,on='YEAR',how='outer')
    return aggregatingdf

In [7]:
# get a list of all the short_desc / fieldnames from which to choose

df_shortdesc = df2[['SHORT_DESC']].drop_duplicates()
df_shortdesc.to_excel(folder+'short_desc_values.xlsx',index=False)

# use this list to find the commodities to focus on. 

In [8]:
#initialize the summary table

print('most recent year in file is',max(df2['YEAR']))
      
new_df = df2[['YEAR']].drop_duplicates()  #.reset_index(drop=True)

new_df = new_df.sort_values(by=['YEAR'])

new_df.reset_index()

print(new_df)


most recent year in file is 2024
          YEAR
713821    1866
1700686   1867
878650    1868
451472    1869
578931    1870
...        ...
51509     2020
416865    2021
2376737   2022
20304134  2023
22172604  2024

[159 rows x 1 columns]


In [9]:
# Now reshape the data so that data points are columns. We'll stick to just the same 159 rows (years)


new_df = append_column(new_df,add_column(df2,'BARLEY - ACRES PLANTED','BARLEY_ACRES'))
new_df = append_column(new_df,add_column(df2,'BARLEY - PRODUCTION, MEASURED IN $','BARLEY_$'))
new_df = append_column(new_df,add_column(df2,'BARLEY - PRODUCTION, MEASURED IN BU','BARLEY_BU  '))
new_df = append_column(new_df,add_column(df2,'BARLEY - YIELD, MEASURED IN BU / ACRE','BARLEY_BU_ACRE'))

new_df = append_column(new_df,add_column(df2,'BLUEBERRIES, TAME - ACRES HARVESTED','_ACRE'))
new_df = append_column(new_df,add_column(df2,'BLUEBERRIES, TAME, UTILIZED - PRODUCTION, MEASURED IN $','BLUEBERRIES  _$'))
new_df = append_column(new_df,add_column(df2,'BLUEBERRIES, TAME, UTILIZED - PRODUCTION, MEASURED IN LB','BLUEBERRIES  _LB'))
new_df = append_column(new_df,add_column(df2,'BLUEBERRIES, TAME, UTILIZED - YIELD, MEASURED IN LB / ACRE','BLUEBERRIES  _LB_ACRE'))

new_df = append_column(new_df,add_column(df2,'CORN - ACRES PLANTED','CORN_ACRES'))
new_df = append_column(new_df,add_column(df2,'CORN, GRAIN - PRODUCTION, MEASURED IN $','CORN_$'))
new_df = append_column(new_df,add_column(df2,'CORN, GRAIN - PRODUCTION, MEASURED IN BU','CORN_BU  '))
new_df = append_column(new_df,add_column(df2,'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE','CORN_BU_ACRE'))
new_df = append_column(new_df,add_column(df2,'COTTON - ACRES PLANTED','COTTON_ACRES'))
new_df = append_column(new_df,add_column(df2,'COTTON - PRODUCTION, MEASURED IN $','COTTON_$'))
new_df = append_column(new_df,add_column(df2,'COTTON - PRODUCTION, MEASURED IN 480 LB BALES','COTTON_LB'))
new_df = append_column(new_df,add_column(df2,'COTTON - YIELD, MEASURED IN LB / ACRE','COTTON_LB_ACRE'))

new_df = append_column(new_df,add_column(df2,'HAY - ACRES HARVESTED','HAY_ACRES'))
new_df = append_column(new_df,add_column(df2,'HAY - PRODUCTION, MEASURED IN $','HAY_$'))
new_df = append_column(new_df,add_column(df2,'HAY - PRODUCTION, MEASURED IN TONS','HAY_T   '))
new_df = append_column(new_df,add_column(df2,'HAY - YIELD, MEASURED IN TONS / ACRE','HAY_T_ACRE'))

new_df = append_column(new_df,add_column(df2,'OATS - ACRES PLANTED','OATS_ACRES'))
new_df = append_column(new_df,add_column(df2,'OATS - PRODUCTION, MEASURED IN $','OATS_$'))
new_df = append_column(new_df,add_column(df2,'OATS - PRODUCTION, MEASURED IN BU','OATS_BU  '))
new_df = append_column(new_df,add_column(df2,'OATS - YIELD, MEASURED IN BU / ACRE','OATS_BU_ACRE'))

new_df = append_column(new_df,add_column(df2,'PEANUTS - ACRES PLANTED','PEANUTS_ACRES'))
new_df = append_column(new_df,add_column(df2,'PEANUTS - PRODUCTION, MEASURED IN $','PEANUTS_$'))
new_df = append_column(new_df,add_column(df2,'PEANUTS - PRODUCTION, MEASURED IN LB','PEANUTS_LB'))
new_df = append_column(new_df,add_column(df2,'PEANUTS - YIELD, MEASURED IN LB / ACRE','PEANUTS_LB_ACRE'))
new_df = append_column(new_df,add_column(df2,'PEPPERS, BELL - ACRES PLANTED','PEPPERS, BELL_ACRES'))
new_df = append_column(new_df,add_column(df2,'PEPPERS, BELL - PRODUCTION, MEASURED IN $','PEPPERS, BELL_$'))
new_df = append_column(new_df,add_column(df2,'PEPPERS, BELL - PRODUCTION, MEASURED IN CWT','PEPPERS, BELL_CWT'))
new_df = append_column(new_df,add_column(df2,'PEPPERS, BELL - YIELD, MEASURED IN CWT / ACRE','PEPPERS, BELL_CWT_ACRE'))


new_df = append_column(new_df,add_column(df2,'PUMPKINS - ACRES PLANTED','PUMPKINS_ACRES'))
new_df = append_column(new_df,add_column(df2,'PUMPKINS - PRODUCTION, MEASURED IN $','PUMPKINS_$'))
new_df = append_column(new_df,add_column(df2,'PUMPKINS - PRODUCTION, MEASURED IN CWT','PUMPKINS_CWT'))
new_df = append_column(new_df,add_column(df2,'PUMPKINS - YIELD, MEASURED IN CWT / ACRE','PUMPKINS_CWT_ACRE'))

new_df = append_column(new_df,add_column(df2,'SOYBEANS - ACRES PLANTED','SOYBEANS_ACRES'))
new_df = append_column(new_df,add_column(df2,'SOYBEANS - PRODUCTION, MEASURED IN $','SOYBEANS_$'))
new_df = append_column(new_df,add_column(df2,'SOYBEANS - PRODUCTION, MEASURED IN BU','SOYBEANS_BU  '))
new_df = append_column(new_df,add_column(df2,'SOYBEANS - YIELD, MEASURED IN BU / ACRE','SOYBEANS_BU_ACRE'))
new_df = append_column(new_df,add_column(df2,'SQUASH - ACRES PLANTED','SQUASH_ACRES'))
new_df = append_column(new_df,add_column(df2,'SQUASH - PRODUCTION, MEASURED IN $','SQUASH_$'))
new_df = append_column(new_df,add_column(df2,'SQUASH - PRODUCTION, MEASURED IN CWT','SQUASH_CWT'))
new_df = append_column(new_df,add_column(df2,'SQUASH - YIELD, MEASURED IN CWT / ACRE','SQUASH_CWT_ACRE'))


new_df = append_column(new_df,add_column(df2,'SWEET POTATOES - ACRES PLANTED','SWEET_ACRES'))
new_df = append_column(new_df,add_column(df2,'SWEET POTATOES - PRODUCTION, MEASURED IN $','SWEET_$'))
new_df = append_column(new_df,add_column(df2,'SWEET POTATOES - PRODUCTION, MEASURED IN CWT','SWEET_CWT'))
new_df = append_column(new_df,add_column(df2,'SWEET POTATOES - YIELD, MEASURED IN CWT / ACRE','SWEET_CWT_ACRE'))
new_df = append_column(new_df,add_column(df2,'TOBACCO - ACRES HARVESTED','TOBACCO_ACRES'))
new_df = append_column(new_df,add_column(df2,'TOBACCO - PRODUCTION, MEASURED IN $','TOBACCO_$'))
new_df = append_column(new_df,add_column(df2,'TOBACCO - PRODUCTION, MEASURED IN LB','TOBACCO_LB'))
new_df = append_column(new_df,add_column(df2,'TOBACCO - YIELD, MEASURED IN LB / ACRE','TOBACCO_LB_ACRE'))

new_df = append_column(new_df,add_column(df2,'WHEAT - ACRES PLANTED','WHEAT_ACRES'))
new_df = append_column(new_df,add_column(df2,'WHEAT - PRODUCTION, MEASURED IN $','WHEAT_$'))
new_df = append_column(new_df,add_column(df2,'WHEAT - PRODUCTION, MEASURED IN BU','WHEAT_BU  '))
new_df = append_column(new_df,add_column(df2,'WHEAT - YIELD, MEASURED IN BU / ACRE','WHEAT_BU_ACRE'))




In [10]:
display(new_df.shape)
display(new_df['YEAR'].value_counts())
new_df.head()

# we now have a table with one row per year, and a lot of columns!

(159, 57)

1866    1
1975    1
1968    1
1969    1
1970    1
       ..
1920    1
1921    1
1922    1
1923    1
2024    1
Name: YEAR, Length: 159, dtype: int64

,YEAR,BARLEY_ACRES,BARLEY_$,BARLEY_BU,BARLEY_BU_ACRE,_ACRE,BLUEBERRIES _$,BLUEBERRIES _LB,BLUEBERRIES _LB_ACRE,CORN_ACRES,CORN_$,CORN_BU,CORN_BU_ACRE,COTTON_ACRES,COTTON_$,COTTON_LB,COTTON_LB_ACRE,HAY_ACRES,HAY_$,HAY_T,HAY_T_ACRE,OATS_ACRES,OATS_$,OATS_BU,OATS_BU_ACRE,PEANUTS_ACRES,PEANUTS_$,PEANUTS_LB,PEANUTS_LB_ACRE,"PEPPERS, BELL_ACRES","PEPPERS, BELL_$","PEPPERS, BELL_CWT","PEPPERS, BELL_CWT_ACRE",PUMPKINS_ACRES,PUMPKINS_$,PUMPKINS_CWT,PUMPKINS_CWT_ACRE,SOYBEANS_ACRES,SOYBEANS_$,SOYBEANS_BU,SOYBEANS_BU_ACRE,SQUASH_ACRES,SQUASH_$,SQUASH_CWT,SQUASH_CWT_ACRE,SWEET_ACRES,SWEET_$,SWEET_CWT,SWEET_CWT_ACRE,TOBACCO_ACRES,TOBACCO_$,TOBACCO_LB,TOBACCO_LB_ACRE,WHEAT_ACRES,WHEAT_$,WHEAT_BU,WHEAT_BU_ACRE
0,1866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15912000.0,9.5,NaN,NaN,109000.0,124.0,NaN,NaN,NaN,NaN,NaN,NaN,2470000.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16000.0,1568000.0,7840000.0,490.0,NaN,NaN,1800000.0,6.0
1,1867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19838000.0,11.5,NaN,NaN,162000.0,129.0,NaN,NaN,NaN,NaN,NaN,NaN,2400000.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19000.0,1550000.0,9120000.0,480.0,NaN,NaN,2240000.0,7.0
2,1868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25650000.0,13.5,NaN,NaN,139000.0,162.0,NaN,NaN,NaN,NaN,NaN,NaN,3308000.0,13.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3263000.0,2016000.0,52.0,20000.0,1536000.0,9200000.0,460.0,NaN,NaN,2250000.0,6.0
3,1869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19000000.0,10.0,NaN,NaN,145000.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN,3510000.0,13.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2706000.0,1691000.0,41.0,26000.0,1509000.0,11180000.0,430.0,NaN,NaN,2925000.0,7.5
4,1870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26245000.0,14.5,NaN,NaN,195000.0,180.0,NaN,NaN,NaN,NaN,NaN,NaN,3640000.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2967000.0,2435000.0,59.0,32000.0,2346000.0,16640000.0,520.0,NaN,NaN,2870000.0,7.0


In [11]:
# verify that all columns have at least some data

new_df.isna().sum()

# drop the 2024 data, which is incomplete
new_df = new_df[new_df['YEAR'] < 2024]


In [12]:
# Now let's save the transformed data

new_df.to_csv(folder+'cropsANNUALNCwide.csv',index=False)